In [1]:

import torch
import time
import random
import numpy as np
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torchvision.models as models
import torchvision.transforms as transforms

from scipy.spatial.distance import cdist
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [2]:


def test_model(model, test_loader, end_id=-1):
    model.eval()
    with torch.no_grad():
        number_corrects = 0
        number_samples = 0
        ss = time.time()
        for i, (test_images_set , test_labels_set) in enumerate(test_loader):
            test_images_set = test_images_set.to(device)
            test_labels_set = test_labels_set.to(device)

            y_predicted = model(test_images_set)
            labels_predicted = y_predicted.argmax(axis = 1)
            number_corrects += (labels_predicted==test_labels_set).sum().item()
            number_samples += test_labels_set.size(0)
            if end_id != -1 and i == end_id: break
            if i%50 == 0:
                ee = time.time()
                print(number_corrects, number_samples, number_corrects/number_samples, ee-ss)
                ss = time.time()
            
        return (number_corrects / number_samples)*100
    
    

def forward_layers(Qs, lambdas, num_Qs):
    
    dist_Qs = np.array([cdist(Qs[Q_id][0],Qs[Q_id][1]) for Q_id in range(num_Qs)])
    weighted_sum_mat = np.sum(dist_Qs*lambdas.reshape(num_Qs,1,1), axis=0)
    return weighted_sum_mat

def rel_position(cur_q, nex_q, Q_dim):
    # cur_q, nex_q are n*d matrices
    res_pos = []
    for q_id in range(cur_q.shape[0]):
        res_pos.append(cur_q[q_id].reshape(1,Q_dim) - nex_q)
    return np.array(res_pos)

def update_params_rigid(cur_coords, nex_coords, lambdas, t_mat, Q_dim, num_Qs, step_size=0.002):
    
    Q_rol = cur_coords.shape[1]
    Q_col = nex_coords.shape[1]
    
    rel_dist_list = np.array([5*(cdist(cur_coords[set_id], nex_coords[set_id])-0.5) for set_id in range(num_Qs)]) 
    rel_vect_list = np.array([rel_position(cur_coords[set_id], nex_coords[set_id], Q_dim) for set_id in range(num_Qs)])
    res_mat = np.sum(rel_dist_list*lambdas.reshape(num_Qs,1,1), axis=0)
    res_error = res_mat - t_mat
    
    for Q_id in range(num_Qs):
        
        cur_lambda = lambdas[Q_id]
        rel_dist_M = step_size*rel_dist_list[Q_id]
        rel_vect_M = step_size*rel_vect_list[Q_id]
        
        resize_error = np.repeat(res_error.reshape(Q_rol,Q_col,1),Q_dim,axis=1).reshape(Q_rol,Q_col,Q_dim)        
        _delta = 2*cur_lambda*rel_vect_M*resize_error#/reg_sum
        cur_coords[Q_id] -= np.sum(_delta,axis=1)
        nex_coords[Q_id] += np.sum(_delta,axis=0)
        
        lambdas[Q_id] -= np.sum(rel_dist_M*res_error)
        
    return cur_coords, nex_coords, lambdas, np.sum(abs(res_error))


def update_params_fluid(cur_coords, nex_coords, lambdas, t_mat, Q_dim, num_Qs, step_size=0.002):
    
    Q_rol = cur_coords.shape[1]
    Q_col = nex_coords.shape[1]
    
    rel_dist_list = np.array([5*(cdist(cur_coords[set_id], nex_coords[set_id])-0.5) for set_id in range(num_Qs)]) 
    rel_vect_list = np.array([rel_position(cur_coords[set_id], nex_coords[set_id], Q_dim) for set_id in range(num_Qs)])
    res_mat = np.sum(rel_dist_list*lambdas.reshape(num_Qs,1,1), axis=0)
    res_error = res_mat - t_mat
    
    for Q_id in range(num_Qs):
        
        cur_lambda = lambdas[Q_id]
        rel_dist_M = step_size*rel_dist_list[Q_id]
        rel_vect_M = step_size*rel_vect_list[Q_id]
        
        resize_error = np.repeat(res_error.reshape(Q_rol,Q_col,1),Q_dim,axis=1).reshape(Q_rol,Q_col,Q_dim)
        _delta = 2*cur_lambda*rel_vect_M*resize_error#/reg_sum
                
        rel_curQs = cdist(cur_coords[Q_id], cur_coords[Q_id])+1
        rel_nexQs = cdist(nex_coords[Q_id], nex_coords[Q_id])+1
        
        cur_coords[Q_id] -= (1/rel_curQs**2)@np.sum(_delta,axis=1)
        nex_coords[Q_id] += (1/rel_nexQs**2)@np.sum(_delta,axis=0)
        
        lambdas[Q_id] -= np.sum(rel_dist_M*res_error)
        
    return cur_coords, nex_coords, lambdas, np.sum(abs(res_error))
        



In [3]:


transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

val_dataset = torchvision.datasets.ImageNet(root= './imagenet_data', split='val', transform = transform)



In [4]:

test_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 32, shuffle = False)

n_total_step = len(test_loader)
print(n_total_step)



1563


In [14]:



def init_dyn(_transDict, _model, _coordsSet={}, change_vars=[], isChange=False):
    
    c_model = copy.deepcopy(_model)
    
    #_coordsSet = {}
    for var_name in _transDict:
        if isChange and var_name not in change_vars: continue
        raw_param = c_model.state_dict()[var_name]
        dynId = var_name.replace('features.','').replace('denseblock','B').replace('conv','C').\
                    replace('denselayer','L').replace('.weight','').replace('.','_')
        #print(dynId, var_name)
        _shape, _config = _transDict[var_name]
        
        for _i in range(_shape[2]):
            for _j in range(_shape[3]):
                if len(_shape) == 6:                    
                    block_i, block_j = _shape[4], _shape[5]

                    for b_i in range(0,_shape[0],block_i):
                        for b_j in range(0,_shape[1],block_j):
                            _subsId = dynId + '_' + str(_i)+'_'+str(_j)+'#'+str(b_i)+'_'+str(b_j)
                            cur_t_mat = raw_param[:,:,_i,_j][b_i:b_i+block_i,b_j:b_j+block_j]
                            
                            _coordsSet[_subsId] = {
                                'i': np.random.rand(_config[0], cur_t_mat.size(0), _config[1]),
                                'h': np.random.rand(_config[0], cur_t_mat.size(1), _config[1]),
                                'ih': np.random.rand(_config[0],1)/(0.5*_config[0]),
                                'var_name': var_name,
                                't_mat': cur_t_mat.numpy(),
                                'step_size': _config[2],
                                'scale': _config[3],
                                'split_ids': [_i, _j],
                                'block_id': [b_i, b_j, block_i, block_j]
                            }

                else:
                    _subsId = dynId + '_' + str(_i)+'_'+str(_j)
                    _coordsSet[_subsId] = {
                        'i': np.random.rand(_config[0], _shape[0], _config[1]),
                        'h': np.random.rand(_config[0], _shape[1], _config[1]),
                        'ih': np.random.rand(_config[0],1)/(0.5*_config[0]),
                        'var_name': var_name,
                        't_mat': raw_param[:,:,_i,_j].numpy(),
                        'step_size': _config[2],
                        'scale': _config[3],
                        'split_ids': [_i, _j],
                        'block_id': []
                    }
        
    return _coordsSet


def dyn_learning(_coordsSet, _transDict, max_epoch=10000, loss_thres=-1):
    
    for _subsId in _coordsSet:
        i_coords = _coordsSet[_subsId]['i']
        h_coords = _coordsSet[_subsId]['h']
        ih_trans = _coordsSet[_subsId]['ih']
        t_mat = _coordsSet[_subsId]['t_mat']
        
        num_Qs, Q_dim, step_size, scale = _transDict[_coordsSet[_subsId]['var_name']][1]
        
        res_error = 0
        for _ep in range(max_epoch):
        
            i_coords, h_coords, ih_trans, _error = update_params_rigid(i_coords, h_coords, ih_trans, t_mat*scale, 
                                                                       Q_dim, num_Qs, step_size=step_size)
            res_error = _error
            if _ep != 0 and _ep % 5000 == 0: print('----- ', _ep, _subsId, _error)
            #if _ep % 500 == 0: print('----- ', _ep, _subsId, _error)
            if _error < loss_thres:
                print('--- Early Stop ', _subsId, _ep, ':', _error)
                break
            
        print(_subsId, res_error)

    return _coordsSet


def recover_weights(cur_coords, nex_coords, lambdas, num_Qs):
    
    rel_dist_list = np.array([5*(cdist(cur_coords[set_id], nex_coords[set_id])-0.5) for set_id in range(num_Qs)]) 
    res_mat = np.sum(rel_dist_list*lambdas.reshape(num_Qs,1,1), axis=0)
    
    return res_mat


def recover_params(_coordsSet, _transDict):
    
    res_dict = {}
    for _subsId in _coordsSet:
        i_coords = _coordsSet[_subsId]['i']
        h_coords = _coordsSet[_subsId]['h']
        ih_trans = _coordsSet[_subsId]['ih']
        var_name = _coordsSet[_subsId]['var_name']
        block_id = _coordsSet[_subsId]['block_id']
        _i, _j = _coordsSet[_subsId]['split_ids']
        _scale = _coordsSet[_subsId]['scale']
        
        if var_name not in res_dict: 
            res_dict[var_name] = np.zeros(_transDict[var_name][0])
            res_dict[var_name] = {
                'rec_param': np.zeros(_transDict[var_name][0][:4]),
                'subsIds': [],
                'param_count': 0
            }
        
        res_dict[var_name]['param_count'] += i_coords.shape[0]*(i_coords.shape[1]+h_coords.shape[1])
        if block_id == []: res_dict[var_name]['rec_param'][:,:,_i,_j] = recover_weights(i_coords, h_coords, ih_trans, _transDict[var_name][1][0])/_scale
        else:
            b_i, b_j, block_i, block_j = block_id
            res_dict[var_name]['rec_param'][b_i:b_i+block_i,b_j:b_j+block_j,_i,_j] = recover_weights(i_coords, h_coords, ih_trans, _transDict[var_name][1][0])/_scale
        
        res_dict[var_name]['subsIds'].append(_subsId)
            
    return res_dict


def recover_dyn(_coordsSet, _transDict, _model, fit_thres=0.1):
    
    before_trans = 0
    after_trans = 0
    
    c_model = copy.deepcopy(_model)
    params_dict = recover_params(_coordsSet, _transDict)
    for var_name in params_dict:
        rec_param = torch.from_numpy(params_dict[var_name]['rec_param'])
        tar_param = _model.state_dict()[var_name]
        
        _shape, _config = _transDict[var_name]
        
        _loss = torch.sum(abs(rec_param-tar_param))/rec_param.numel()
        rel_fitness = (_loss/torch.var(tar_param)).item()
        print(var_name, rel_fitness)
        if 1==1 and rel_fitness < fit_thres:
            c_model.state_dict()[var_name].copy_(rec_param)
            before_trans += tar_param.numel()
            #after_trans += (_shape[0]+_shape[1])*_shape[2]*_shape[3]*_config[0]
            after_trans += params_dict[var_name]['param_count']
            
    print(before_trans, after_trans)
                    
    return c_model





In [5]:

#vgg_model = models.vgg16(pretrained=True)
resnet_model = models.resnet50(pretrained=True)
#densenet_model = models.densenet121(pretrained=True)


In [6]:

resnet_transDict = {}
total_params = 0
    
for name, param in resnet_model.state_dict().items():
    if len(param.size()) > 1:
        print(name, list(param.size()), param.numel(),round(0.5/torch.var(param).item()))
        list_param = list(param.size())
        if list_param[0] >= 128 and list_param[1] >= 128:
            resnet_transDict[name] = [list_param+[128,128],[5,35,5e-7,round(0.5/torch.var(param).item())]]
        elif list_param[0] > 64 or list_param[1] > 64:
            resnet_transDict[name] = [list_param+[64,64],[5,18,1e-6,round(0.5/torch.var(param).item())]]
        else: resnet_transDict[name] = [list_param,[5,15,1e-6,round(0.5/torch.var(param).item())]]
        
    total_params += param.numel()
    
print(total_params)


conv1.weight [64, 3, 7, 7] 9408 33
layer1.0.conv1.weight [64, 64, 1, 1] 4096 100
layer1.0.conv2.weight [64, 64, 3, 3] 36864 588
layer1.0.conv3.weight [256, 64, 1, 1] 16384 402
layer1.0.downsample.0.weight [256, 64, 1, 1] 16384 151
layer1.1.conv1.weight [64, 256, 1, 1] 16384 533
layer1.1.conv2.weight [64, 64, 3, 3] 36864 602
layer1.1.conv3.weight [256, 64, 1, 1] 16384 472
layer1.2.conv1.weight [64, 256, 1, 1] 16384 553
layer1.2.conv2.weight [64, 64, 3, 3] 36864 495
layer1.2.conv3.weight [256, 64, 1, 1] 16384 517
layer2.0.conv1.weight [128, 256, 1, 1] 32768 399
layer2.0.conv2.weight [128, 128, 3, 3] 147456 1006
layer2.0.conv3.weight [512, 128, 1, 1] 65536 634
layer2.0.downsample.0.weight [512, 256, 1, 1] 131072 936
layer2.1.conv1.weight [128, 512, 1, 1] 65536 1802
layer2.1.conv2.weight [128, 128, 3, 3] 147456 1337
layer2.1.conv3.weight [512, 128, 1, 1] 65536 1028
layer2.2.conv1.weight [128, 512, 1, 1] 65536 920
layer2.2.conv2.weight [128, 128, 3, 3] 147456 1072
layer2.2.conv3.weight [512

In [7]:

resnet_transDict


{'conv1.weight': [[64, 3, 7, 7], [5, 15, 1e-06, 33]],
 'layer1.0.conv1.weight': [[64, 64, 1, 1], [5, 15, 1e-06, 100]],
 'layer1.0.conv2.weight': [[64, 64, 3, 3], [5, 15, 1e-06, 588]],
 'layer1.0.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 402]],
 'layer1.0.downsample.0.weight': [[256, 64, 1, 1, 64, 64],
  [5, 18, 1e-06, 151]],
 'layer1.1.conv1.weight': [[64, 256, 1, 1, 64, 64], [5, 18, 1e-06, 533]],
 'layer1.1.conv2.weight': [[64, 64, 3, 3], [5, 15, 1e-06, 602]],
 'layer1.1.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 472]],
 'layer1.2.conv1.weight': [[64, 256, 1, 1, 64, 64], [5, 18, 1e-06, 553]],
 'layer1.2.conv2.weight': [[64, 64, 3, 3], [5, 15, 1e-06, 495]],
 'layer1.2.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 517]],
 'layer2.0.conv1.weight': [[128, 256, 1, 1, 128, 128], [5, 35, 5e-07, 399]],
 'layer2.0.conv2.weight': [[128, 128, 3, 3, 128, 128], [5, 35, 5e-07, 1006]],
 'layer2.0.conv3.weight': [[512, 128, 1, 1, 128, 128], [5, 35, 5e-07, 634]],
 

In [12]:

resnet_transDict = {
    'conv1.weight': [[64, 3, 7, 7], [3, 5, 1e-5, 66]],
    'layer1.0.conv1.weight': [[64, 64, 1, 1], [5, 18, 1e-06, 100]],
    'layer1.0.conv2.weight': [[64, 64, 3, 3], [5, 18, 1e-06, 588]],
    'layer1.0.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 402]],
    'layer1.0.downsample.0.weight': [[256, 64, 1, 1, 64, 64],[5, 18, 1e-06, 151]],
    'layer1.1.conv1.weight': [[64, 256, 1, 1, 64, 64], [5, 18, 1e-06, 533]],
    'layer1.1.conv2.weight': [[64, 64, 3, 3], [5, 18, 1e-06, 602]],
    'layer1.1.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 472]],
    'layer1.2.conv1.weight': [[64, 256, 1, 1, 64, 64], [5, 18, 1e-06, 553]],
    'layer1.2.conv2.weight': [[64, 64, 3, 3], [5, 18, 1e-06, 495]],
    'layer1.2.conv3.weight': [[256, 64, 1, 1, 64, 64], [5, 18, 1e-06, 517]],
    'layer2.0.conv1.weight': [[128, 256, 1, 1, 128, 128], [5, 35, 5e-07, 399]],
    'layer2.0.conv2.weight': [[128, 128, 3, 3, 128, 128], [5, 35, 5e-07, 1006]],
    'layer2.0.conv3.weight': [[512, 128, 1, 1, 128, 128], [6, 40, 2e-07, 634]],
    'layer2.0.downsample.0.weight': [[512, 256, 1, 1, 128, 128],[6, 40, 2e-07, 936]],
    'layer2.1.conv1.weight': [[128, 512, 1, 1, 128, 128], [6, 40, 2e-07, 1802]],
    'layer2.1.conv2.weight': [[128, 128, 3, 3, 128, 128], [6, 40, 2e-07, 1337]],
    'layer2.1.conv3.weight': [[512, 128, 1, 1, 128, 128], [6, 40, 2e-07, 1028]],
    'layer2.2.conv1.weight': [[128, 512, 1, 1, 128, 128], [5, 35, 5e-07, 920]],
    'layer2.2.conv2.weight': [[128, 128, 3, 3, 128, 128], [5, 35, 5e-07, 1072]],
    'layer2.2.conv3.weight': [[512, 128, 1, 1, 128, 128], [5, 35, 5e-07, 730]],
    'layer2.3.conv1.weight': [[128, 512, 1, 1, 128, 128], [5, 35, 5e-07, 846]],
    'layer2.3.conv2.weight': [[128, 128, 3, 3, 128, 128], [5, 35, 5e-07, 996]],
    'layer2.3.conv3.weight': [[512, 128, 1, 1, 128, 128], [5, 35, 5e-07, 846]],
    'layer3.0.conv1.weight': [[256, 512, 1, 1, 128, 128], [5, 35, 5e-07, 528]],
    'layer3.0.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 1659]],
    'layer3.0.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 915]],
    'layer3.0.downsample.0.weight': [[1024, 512, 1, 1, 128, 128],[5, 35, 5e-07, 1917]],
    'layer3.1.conv1.weight': [[256, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 2160]],
    'layer3.1.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 2194]],
    'layer3.1.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 1207]],
    'layer3.2.conv1.weight': [[256, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 2016]],
    'layer3.2.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 2221]],
    'layer3.2.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 1374]],
    'layer3.3.conv1.weight': [[256, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 1662]],
    'layer3.3.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 2244]],
    'layer3.3.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 1508]],
    'layer3.4.conv1.weight': [[256, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 1524]],
    'layer3.4.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 2261]],
    'layer3.4.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 1517]],
    'layer3.5.conv1.weight': [[256, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 1288]],
    'layer3.5.conv2.weight': [[256, 256, 3, 3, 128, 128], [5, 35, 5e-07, 2152]],
    'layer3.5.conv3.weight': [[1024, 256, 1, 1, 128, 128], [5, 35, 5e-07, 1351]],
    'layer4.0.conv1.weight': [[512, 1024, 1, 1, 128, 128], [5, 35, 5e-07, 883]],
    'layer4.0.conv2.weight': [[512, 512, 3, 3, 128, 128], [5, 35, 5e-07, 3172]],
    'layer4.0.conv3.weight': [[2048, 512, 1, 1, 128, 128], [5, 35, 5e-07, 2138]],
    'layer4.0.downsample.0.weight': [[2048, 1024, 1, 1, 128, 128],[5, 35, 5e-07, 5065]],
    'layer4.1.conv1.weight': [[512, 2048, 1, 1, 128, 128], [5, 35, 5e-07, 2307]],
    'layer4.1.conv2.weight': [[512, 512, 3, 3, 128, 128], [5, 35, 5e-07, 3307]],
    'layer4.1.conv3.weight': [[2048, 512, 1, 1, 128, 128], [5, 35, 5e-07, 2237]],
    'layer4.2.conv1.weight': [[512, 2048, 1, 1, 128, 128], [5, 35, 5e-07, 1521]],
    'layer4.2.conv2.weight': [[512, 512, 3, 3, 128, 128], [5, 35, 5e-07, 4099]],
    'layer4.2.conv3.weight': [[2048, 512, 1, 1, 128, 128], [5, 35, 5e-07, 2489]]
}



In [10]:

dyn_coords = init_dyn(resnet_transDict, resnet_model)


In [11]:

ss = time.time()
dyn_coords = dyn_learning(dyn_coords, resnet_transDict, max_epoch=30000, loss_thres=1)
trans_model = recover_dyn(dyn_coords, resnet_transDict, densenet_model)
ee = time.time()
print('Time Cost:', ee-ss)

test_res = test_model(trans_model, test_loader, end_id=-1)
print('All Accuracy:', test_res)

date_id = 'jul30'
np.save('./imagenet_coords/resnet_coords/R_'+date_id+'_S0-5_1_'+str(round(test_res*1000))+'.npy', dyn_coords)



-----  5000 C1_0_0 244.06374179341853
-----  10000 C1_0_0 171.0896234961204
-----  15000 C1_0_0 84.57963858454985
-----  20000 C1_0_0 34.94932810389784
-----  25000 C1_0_0 14.818803442957584
C1_0_0 6.782511784432142
-----  5000 C1_0_1 213.40338162471903
-----  10000 C1_0_1 154.39597599969085
-----  15000 C1_0_1 100.05998735815263
-----  20000 C1_0_1 61.34554448446146
-----  25000 C1_0_1 44.42903853097845
C1_0_1 32.69143326526732
-----  5000 C1_0_2 208.46541285553425
-----  10000 C1_0_2 125.87615140440519
-----  15000 C1_0_2 97.35712918595132
-----  20000 C1_0_2 81.38703541370538
-----  25000 C1_0_2 61.30453939843573
C1_0_2 40.47087653072433
-----  5000 C1_0_3 198.1861900122745
-----  10000 C1_0_3 79.24457331127083
-----  15000 C1_0_3 40.144029223874725
-----  20000 C1_0_3 20.1687483302809
-----  25000 C1_0_3 10.202055875935239
C1_0_3 5.251241228048977
-----  5000 C1_0_4 188.98508301174445
-----  10000 C1_0_4 87.86376231440144
-----  15000 C1_0_4 62.00344494515249
-----  20000 C1_0_4 40

-----  10000 layer1_0_C2_0_0 1926.1781738971256
-----  15000 layer1_0_C2_0_0 965.8683858902377
-----  20000 layer1_0_C2_0_0 560.4642434134869
-----  25000 layer1_0_C2_0_0 355.14443312610535
layer1_0_C2_0_0 240.95893568772686
-----  5000 layer1_0_C2_0_1 3289.4250053870114
-----  10000 layer1_0_C2_0_1 1179.4092935148205
-----  15000 layer1_0_C2_0_1 628.6320751270753
-----  20000 layer1_0_C2_0_1 386.7895304910875
-----  25000 layer1_0_C2_0_1 255.86662404324622
layer1_0_C2_0_1 176.70981025695895
-----  5000 layer1_0_C2_0_2 5231.997908498568
-----  10000 layer1_0_C2_0_2 1851.9320389504337
-----  15000 layer1_0_C2_0_2 1054.99650216517
-----  20000 layer1_0_C2_0_2 721.6163399455695
-----  25000 layer1_0_C2_0_2 524.6554995254485
layer1_0_C2_0_2 392.38898191794476
-----  5000 layer1_0_C2_1_0 4519.538636901114
-----  10000 layer1_0_C2_1_0 2280.1503374909353
-----  15000 layer1_0_C2_1_0 1515.610275698712
-----  20000 layer1_0_C2_1_0 1121.3785203915932
-----  25000 layer1_0_C2_1_0 877.830257805873

-----  15000 layer1_1_C2_2_1 247.99386845086366
-----  20000 layer1_1_C2_2_1 114.21165158447957
-----  25000 layer1_1_C2_2_1 57.94132669913523
layer1_1_C2_2_1 31.349767020031813
-----  5000 layer1_1_C2_2_2 4106.767001357802
-----  10000 layer1_1_C2_2_2 1228.194261037879
-----  15000 layer1_1_C2_2_2 549.9177011278467
-----  20000 layer1_1_C2_2_2 305.397326619728
-----  25000 layer1_1_C2_2_2 191.48660965507287
layer1_1_C2_2_2 127.95787220191522
-----  5000 layer1_1_C3_0_0#0_0 3106.4313825042873
-----  10000 layer1_1_C3_0_0#0_0 1012.9791621218069
-----  15000 layer1_1_C3_0_0#0_0 520.5628263893527
-----  20000 layer1_1_C3_0_0#0_0 315.4984489690858
-----  25000 layer1_1_C3_0_0#0_0 206.98139043732584
layer1_1_C3_0_0#0_0 142.24510213013045
-----  5000 layer1_1_C3_0_0#64_0 2574.744472320683
-----  10000 layer1_1_C3_0_0#64_0 674.0330493644308
-----  15000 layer1_1_C3_0_0#64_0 279.0658057296039
-----  20000 layer1_1_C3_0_0#64_0 133.2694295083515
-----  25000 layer1_1_C3_0_0#64_0 69.4073963013577

layer2_0_C2_1_0#0_0 96.76032391182423
-----  5000 layer2_0_C2_1_1#0_0 12044.795238403307
-----  10000 layer2_0_C2_1_1#0_0 3638.4886567629765
-----  15000 layer2_0_C2_1_1#0_0 1392.9670829151498
-----  20000 layer2_0_C2_1_1#0_0 588.8898395178026
-----  25000 layer2_0_C2_1_1#0_0 273.8959373040957
layer2_0_C2_1_1#0_0 140.20481593700816
-----  5000 layer2_0_C2_1_2#0_0 14526.140552443183
-----  10000 layer2_0_C2_1_2#0_0 3168.8807556621805
-----  15000 layer2_0_C2_1_2#0_0 1073.5083937003687
-----  20000 layer2_0_C2_1_2#0_0 516.0453880432808
-----  25000 layer2_0_C2_1_2#0_0 325.3387484308506
layer2_0_C2_1_2#0_0 223.38121473619765
-----  5000 layer2_0_C2_2_0#0_0 9272.744283349264
-----  10000 layer2_0_C2_2_0#0_0 1840.7810874276133
-----  15000 layer2_0_C2_2_0#0_0 539.519309259328
-----  20000 layer2_0_C2_2_0#0_0 201.4474846125746
-----  25000 layer2_0_C2_2_0#0_0 103.6737637518932
layer2_0_C2_2_0#0_0 63.83463310144302
-----  5000 layer2_0_C2_2_1#0_0 12748.756868846278
-----  10000 layer2_0_C2_2_

-----  20000 layer2_1_C2_1_1#0_0 53514.41592575745
-----  25000 layer2_1_C2_1_1#0_0 54196.826836625536
layer2_1_C2_1_1#0_0 54394.94439653685
-----  5000 layer2_1_C2_1_2#0_0 11008.364671494135
-----  10000 layer2_1_C2_1_2#0_0 92696.13919852485
-----  15000 layer2_1_C2_1_2#0_0 98820.09761397251
-----  20000 layer2_1_C2_1_2#0_0 98175.59757316744
-----  25000 layer2_1_C2_1_2#0_0 97096.14092567409
layer2_1_C2_1_2#0_0 95216.51890069261
-----  5000 layer2_1_C2_2_0#0_0 10573.183090345243
-----  10000 layer2_1_C2_2_0#0_0 3048.455567461737
-----  15000 layer2_1_C2_2_0#0_0 1269.160138121686
-----  20000 layer2_1_C2_2_0#0_0 631.0084052211182
-----  25000 layer2_1_C2_2_0#0_0 348.35305000149265
layer2_1_C2_2_0#0_0 206.13056475605944
-----  5000 layer2_1_C2_2_1#0_0 20821.032959108794
-----  10000 layer2_1_C2_2_1#0_0 8464.554694478808
-----  15000 layer2_1_C2_2_1#0_0 4558.764225846528
-----  20000 layer2_1_C2_2_1#0_0 2805.246463104016
-----  25000 layer2_1_C2_2_1#0_0 1839.8391776141216
layer2_1_C2_2_1

-----  5000 layer2_3_C1_0_0#0_256 10966.362167933734
-----  10000 layer2_3_C1_0_0#0_256 1732.687937272857
-----  15000 layer2_3_C1_0_0#0_256 387.2984999128179
-----  20000 layer2_3_C1_0_0#0_256 102.17189134586226
-----  25000 layer2_3_C1_0_0#0_256 32.11784287652367
layer2_3_C1_0_0#0_256 12.028504165368458
-----  5000 layer2_3_C1_0_0#0_384 8302.456510717304
-----  10000 layer2_3_C1_0_0#0_384 1233.1727317240066
-----  15000 layer2_3_C1_0_0#0_384 245.66323374144844
-----  20000 layer2_3_C1_0_0#0_384 60.23242135864445
-----  25000 layer2_3_C1_0_0#0_384 17.020766814477057
layer2_3_C1_0_0#0_384 5.381777713852372
-----  5000 layer2_3_C2_0_0#0_0 5316.134370067341
-----  10000 layer2_3_C2_0_0#0_0 756.272901301375
-----  15000 layer2_3_C2_0_0#0_0 137.70853555870752
-----  20000 layer2_3_C2_0_0#0_0 35.50450261246928
-----  25000 layer2_3_C2_0_0#0_0 12.912220673432014
layer2_3_C2_0_0#0_0 5.645641391332648
-----  5000 layer2_3_C2_0_1#0_0 9481.059482160374
-----  10000 layer2_3_C2_0_1#0_0 1601.08874

KeyboardInterrupt: 

In [15]:

change_vars = []

dyn_coords = init_dyn(resnet_transDict, resnet_model, _coordsSet=dyn_coords, change_vars=change_vars)


In [ ]:


ss = time.time()
dyn_coords = dyn_learning(dyn_coords, resnet_transDict, max_epoch=30000, loss_thres=200)
trans_model = recover_dyn(dyn_coords, resnet_transDict, densenet_model)
ee = time.time()
print('Time Cost:', ee-ss)

test_res = test_model(trans_model, test_loader, end_id=-1)
print('All Accuracy:', test_res)

date_id = 'jul31'
np.save('./imagenet_coords/resnet_coords/R_'+date_id+'_S200_'+str(round(test_res*1000))+'.npy', dyn_coords)


--- Early Stop  C1_0_0 0 : 6.7794483521801645
C1_0_0 6.7794483521801645
--- Early Stop  C1_0_1 0 : 32.68489921497765
C1_0_1 32.68489921497765
--- Early Stop  C1_0_2 0 : 40.459483376073706
C1_0_2 40.459483376073706
--- Early Stop  C1_0_3 0 : 5.249179821137014
C1_0_3 5.249179821137014
--- Early Stop  C1_0_4 0 : 14.534352983719284
C1_0_4 14.534352983719284
--- Early Stop  C1_0_5 0 : 15.629564332261204
C1_0_5 15.629564332261204
--- Early Stop  C1_0_6 0 : 14.206083952553463
C1_0_6 14.206083952553463
--- Early Stop  C1_1_0 0 : 10.686345889722066
C1_1_0 10.686345889722066
--- Early Stop  C1_1_1 0 : 2.8410576174815785
C1_1_1 2.8410576174815785
--- Early Stop  C1_1_2 0 : 2.027417994808893
C1_1_2 2.027417994808893
--- Early Stop  C1_1_3 0 : 0.9993083308238302
C1_1_3 0.9993083308238302
--- Early Stop  C1_1_4 0 : 0.9986182560446172
C1_1_4 0.9986182560446172
--- Early Stop  C1_1_5 0 : 0.9991561073414305
C1_1_5 0.9991561073414305
--- Early Stop  C1_1_6 0 : 2.802766671124693
C1_1_6 2.802766671124693


--- Early Stop  layer2_0_C1_0_0#0_128 0 : 1.3482932495783055
layer2_0_C1_0_0#0_128 1.3482932495783055
--- Early Stop  layer2_0_C2_0_0#0_0 0 : 14.675145876314513
layer2_0_C2_0_0#0_0 14.675145876314513
--- Early Stop  layer2_0_C2_0_1#0_0 0 : 199.97016237548127
layer2_0_C2_0_1#0_0 199.97016237548127
--- Early Stop  layer2_0_C2_0_2#0_0 0 : 49.9779731526818
layer2_0_C2_0_2#0_0 49.9779731526818
--- Early Stop  layer2_0_C2_1_0#0_0 0 : 96.73596451533894
layer2_0_C2_1_0#0_0 96.73596451533894
--- Early Stop  layer2_0_C2_1_1#0_0 0 : 140.15285437574786
layer2_0_C2_1_1#0_0 140.15285437574786
--- Early Stop  layer2_0_C2_1_2#0_0 0 : 199.96791471607585
layer2_0_C2_1_2#0_0 199.96791471607585
--- Early Stop  layer2_0_C2_2_0#0_0 0 : 63.81761284065091
layer2_0_C2_2_0#0_0 63.81761284065091
--- Early Stop  layer2_0_C2_2_1#0_0 0 : 171.5894140676616
layer2_0_C2_2_1#0_0 171.5894140676616
--- Early Stop  layer2_0_C2_2_2#0_0 0 : 199.98512484469592
layer2_0_C2_2_2#0_0 199.98512484469592
--- Early Stop  layer2_0_C

layer2_2_C2_1_2#0_0 45039.572592426586
--- Early Stop  layer2_2_C2_2_0#0_0 0 : 47.847503748465996
layer2_2_C2_2_0#0_0 47.847503748465996
-----  5000 layer2_2_C2_2_1#0_0 37247.37434323349
-----  10000 layer2_2_C2_2_1#0_0 37239.6488762294
-----  15000 layer2_2_C2_2_1#0_0 37230.78008221336
-----  20000 layer2_2_C2_2_1#0_0 37220.65813012529
-----  25000 layer2_2_C2_2_1#0_0 37209.14727962141
layer2_2_C2_2_1#0_0 37489.35794358389
--- Early Stop  layer2_2_C2_2_2#0_0 0 : 77.22979012297945
layer2_2_C2_2_2#0_0 77.22979012297945
--- Early Stop  layer2_2_C3_0_0#0_0 0 : 12.165914208128696
layer2_2_C3_0_0#0_0 12.165914208128696
--- Early Stop  layer2_2_C3_0_0#128_0 0 : 199.97355628564583
layer2_2_C3_0_0#128_0 199.97355628564583
--- Early Stop  layer2_2_C3_0_0#256_0 0 : 4.536862650409269
layer2_2_C3_0_0#256_0 4.536862650409269
--- Early Stop  layer2_2_C3_0_0#384_0 0 : 1.206015427429593
layer2_2_C3_0_0#384_0 1.206015427429593
--- Early Stop  layer2_3_C1_0_0#0_0 0 : 13.350094424881302
layer2_3_C1_0_0#0

-----  5000 layer3_0_C2_1_1#0_128 170356.895588423
-----  10000 layer3_0_C2_1_1#0_128 167642.64692685116
-----  15000 layer3_0_C2_1_1#0_128 151881.71157888128
-----  20000 layer3_0_C2_1_1#0_128 163544.23400214812
-----  25000 layer3_0_C2_1_1#0_128 207921.97883263836
layer3_0_C2_1_1#0_128 208880.68484551983
-----  5000 layer3_0_C2_1_1#128_0 205398.65466216306
-----  10000 layer3_0_C2_1_1#128_0 141765.93164055664
-----  15000 layer3_0_C2_1_1#128_0 139428.3457499599
-----  20000 layer3_0_C2_1_1#128_0 135811.24947837676
-----  25000 layer3_0_C2_1_1#128_0 125283.4315527801
layer3_0_C2_1_1#128_0 103684.87660433346
-----  5000 layer3_0_C2_1_1#128_128 155621.3041442345
-----  10000 layer3_0_C2_1_1#128_128 160970.82860804186
-----  15000 layer3_0_C2_1_1#128_128 158672.30753617003
-----  20000 layer3_0_C2_1_1#128_128 153683.88640381466
-----  25000 layer3_0_C2_1_1#128_128 150555.20820440835
layer3_0_C2_1_1#128_128 147987.51801023335
-----  5000 layer3_0_C2_1_2#0_0 190853.993176386
-----  10000 l

-----  15000 layer3_0_C3_0_0#512_0 843.5507370264684
-----  20000 layer3_0_C3_0_0#512_0 402.6379781213566
--- Early Stop  layer3_0_C3_0_0#512_0 24603 : 199.96663298014363
layer3_0_C3_0_0#512_0 199.96663298014363
-----  5000 layer3_0_C3_0_0#512_128 13068.734971472879
-----  10000 layer3_0_C3_0_0#512_128 2130.6439332599666
-----  15000 layer3_0_C3_0_0#512_128 498.31465060555706
--- Early Stop  layer3_0_C3_0_0#512_128 18753 : 199.96395689639428
layer3_0_C3_0_0#512_128 199.96395689639428
-----  5000 layer3_0_C3_0_0#640_0 11174.895889449992
-----  10000 layer3_0_C3_0_0#640_0 1645.5041091446928
-----  15000 layer3_0_C3_0_0#640_0 299.95853245312173
--- Early Stop  layer3_0_C3_0_0#640_0 16278 : 199.97935408159154
layer3_0_C3_0_0#640_0 199.97935408159154
-----  5000 layer3_0_C3_0_0#640_128 12463.893870561935
-----  10000 layer3_0_C3_0_0#640_128 2049.9208153832183
-----  15000 layer3_0_C3_0_0#640_128 441.11256434331676
--- Early Stop  layer3_0_C3_0_0#640_128 18218 : 199.97823581841197
layer3_0_C

...

-----  25000 layer3_4_C1_0_0#128_896 129998.94289822815
layer3_4_C1_0_0#128_896 119201.2640241376
-----  5000 layer3_4_C2_0_0#0_0 196099.193311364
-----  10000 layer3_4_C2_0_0#0_0 193981.15929135805
-----  15000 layer3_4_C2_0_0#0_0 179469.0549878024
-----  20000 layer3_4_C2_0_0#0_0 219838.98630726052
-----  25000 layer3_4_C2_0_0#0_0 245839.74793813203
layer3_4_C2_0_0#0_0 244143.69852748996
-----  5000 layer3_4_C2_0_0#0_128 207901.23476545373
-----  10000 layer3_4_C2_0_0#0_128 204699.35706543713
-----  15000 layer3_4_C2_0_0#0_128 240472.96779105216
-----  20000 layer3_4_C2_0_0#0_128 259238.8812580538
-----  25000 layer3_4_C2_0_0#0_128 263318.58114011365
layer3_4_C2_0_0#0_128 260164.81516443542
-----  5000 layer3_4_C2_0_0#128_0 267397.5478606765
-----  10000 layer3_4_C2_0_0#128_0 190404.69687997294
-----  15000 layer3_4_C2_0_0#128_0 146457.2204895701
-----  20000 layer3_4_C2_0_0#128_0 117370.87683400905
-----  25000 layer3_4_C2_0_0#128_0 104445.95440827709
layer3_4_C2_0_0#128_0 99089.714

-----  15000 layer3_4_C2_2_0#128_0 212924.6717596457
-----  20000 layer3_4_C2_2_0#128_0 182980.902705995
-----  25000 layer3_4_C2_2_0#128_0 177632.57153763034
layer3_4_C2_2_0#128_0 207860.9309659473
-----  5000 layer3_4_C2_2_0#128_128 219518.89167670155
-----  10000 layer3_4_C2_2_0#128_128 199604.68118715606
-----  15000 layer3_4_C2_2_0#128_128 238284.72878778924
-----  20000 layer3_4_C2_2_0#128_128 272919.6866067852
-----  25000 layer3_4_C2_2_0#128_128 273628.9359149921
layer3_4_C2_2_0#128_128 268340.5189350096
-----  5000 layer3_4_C2_2_1#0_0 216978.88599909458
-----  10000 layer3_4_C2_2_1#0_0 212997.51790413956
-----  15000 layer3_4_C2_2_1#0_0 241925.37528131335
-----  20000 layer3_4_C2_2_1#0_0 216381.66330863023
-----  25000 layer3_4_C2_2_1#0_0 226181.2929104832
layer3_4_C2_2_1#0_0 195844.32467676347
-----  5000 layer3_4_C2_2_1#0_128 161378.44939449232
-----  10000 layer3_4_C2_2_1#0_128 160049.08625679932
-----  15000 layer3_4_C2_2_1#0_128 152278.6285559574
-----  20000 layer3_4_C2_

layer3_5_C1_0_0#0_0 97077.70396773209
-----  5000 layer3_5_C1_0_0#0_128 51464.18229127408
-----  10000 layer3_5_C1_0_0#0_128 111252.07541432878
-----  15000 layer3_5_C1_0_0#0_128 110550.63164723644
-----  20000 layer3_5_C1_0_0#0_128 110445.65858625375
-----  25000 layer3_5_C1_0_0#0_128 110157.72003403364
layer3_5_C1_0_0#0_128 112094.80915104586
-----  5000 layer3_5_C1_0_0#0_256 20899.734381951173
-----  10000 layer3_5_C1_0_0#0_256 98682.43353635361
-----  15000 layer3_5_C1_0_0#0_256 95213.62679736716
-----  20000 layer3_5_C1_0_0#0_256 95257.65868204916
-----  25000 layer3_5_C1_0_0#0_256 95225.3076213838
layer3_5_C1_0_0#0_256 93345.98910178413
-----  5000 layer3_5_C1_0_0#0_384 64938.34516067333
-----  10000 layer3_5_C1_0_0#0_384 88814.80638716089
-----  15000 layer3_5_C1_0_0#0_384 80890.75314086732
-----  20000 layer3_5_C1_0_0#0_384 73825.22558705388
-----  25000 layer3_5_C1_0_0#0_384 66278.6617627081
layer3_5_C1_0_0#0_384 58081.82258767825
-----  5000 layer3_5_C1_0_0#0_512 18532.790560

-----  20000 layer3_5_C2_0_2#128_128 188200.23147740337
-----  25000 layer3_5_C2_0_2#128_128 235300.57396646033
layer3_5_C2_0_2#128_128 243481.47391654894
-----  5000 layer3_5_C2_1_0#0_0 144549.6670341425
-----  10000 layer3_5_C2_1_0#0_0 156906.6442810992
-----  15000 layer3_5_C2_1_0#0_0 154337.6044559851
-----  20000 layer3_5_C2_1_0#0_0 143655.97104624615
-----  25000 layer3_5_C2_1_0#0_0 143346.98213050253
layer3_5_C2_1_0#0_0 174253.33482493937
-----  5000 layer3_5_C2_1_0#0_128 149305.68768726985
-----  10000 layer3_5_C2_1_0#0_128 151345.78171592607
-----  15000 layer3_5_C2_1_0#0_128 148984.477850869
-----  20000 layer3_5_C2_1_0#0_128 146992.99772709358
-----  25000 layer3_5_C2_1_0#0_128 153014.4877284909
layer3_5_C2_1_0#0_128 155769.66523348348
-----  5000 layer3_5_C2_1_0#128_0 136882.60198020618
-----  10000 layer3_5_C2_1_0#128_0 154058.80944643583
-----  15000 layer3_5_C2_1_0#128_0 152794.6691154893
-----  20000 layer3_5_C2_1_0#128_0 146640.52261257486
-----  25000 layer3_5_C2_1_0#

-----  15000 layer3_5_C3_0_0#128_0 116910.58473904245
-----  20000 layer3_5_C3_0_0#128_0 116254.6789572075
-----  25000 layer3_5_C3_0_0#128_0 114517.78782850338
layer3_5_C3_0_0#128_0 113067.1649261009
-----  5000 layer3_5_C3_0_0#128_128 69118.89576692301
-----  10000 layer3_5_C3_0_0#128_128 132301.99279074973
-----  15000 layer3_5_C3_0_0#128_128 137422.4563019119
-----  20000 layer3_5_C3_0_0#128_128 139152.27898796907
-----  25000 layer3_5_C3_0_0#128_128 142590.6856782394
layer3_5_C3_0_0#128_128 143927.3004103792
-----  5000 layer3_5_C3_0_0#256_0 28558.362756815997
-----  10000 layer3_5_C3_0_0#256_0 111177.84107698155
-----  15000 layer3_5_C3_0_0#256_0 111154.83393955267
-----  20000 layer3_5_C3_0_0#256_0 110923.27646420445
-----  25000 layer3_5_C3_0_0#256_0 110307.55406891782
layer3_5_C3_0_0#256_0 111408.82002510273
-----  5000 layer3_5_C3_0_0#256_128 38928.09926424472
-----  10000 layer3_5_C3_0_0#256_128 120352.07765793691
-----  15000 layer3_5_C3_0_0#256_128 120084.69250494902
-----

In [19]:

np.save('./imagenet_coords/resnet_coords/tmp_aug3.npy', dyn_coords)


In [18]:

trans_model = recover_dyn(dyn_coords, resnet_transDict, resnet_model, fit_thres=0.1)


conv1.weight 0.007833514917757738
layer1.0.conv1.weight 0.037817649398636584
layer1.0.conv2.weight 19.608990036859005
layer1.0.conv3.weight 18.245101133357895
layer1.0.downsample.0.weight 11.46462502827135
layer1.1.conv1.weight 22.7256286573476
layer1.1.conv2.weight 24.336839840684856
layer1.1.conv3.weight 20.058046807724594
layer1.2.conv1.weight 24.277574830824918
layer1.2.conv2.weight 23.969671027633694
layer1.2.conv3.weight 20.330665619533317
layer2.0.conv1.weight 20.295307381090886
layer2.0.conv2.weight 33.79048018555659
layer2.0.conv3.weight 22.918344852461512
layer2.0.downsample.0.weight 26.155510651508987
layer2.1.conv1.weight 36.958555266624664
layer2.1.conv2.weight 33.59596757692909
layer2.1.conv3.weight 26.37562249674676
layer2.2.conv1.weight 30.283788864310534
layer2.2.conv2.weight 33.28177954361143
layer2.2.conv3.weight 27.27877091245327
layer2.3.conv1.weight 30.984389429399346
layer2.3.conv2.weight 34.11307747237198
layer2.3.conv3.weight 29.033923637887582
layer3.0.conv1.w

In [19]:

test_res = test_model(trans_model, test_loader, end_id=-1)
print(test_res)


29 32 0.90625 10.189950942993164
1400 1632 0.8578431372549019 454.3236548900604
2538 3232 0.7852722772277227 416.20040488243103
3864 4832 0.7996688741721855 394.3900911808014
5155 6432 0.8014614427860697 389.58656787872314
6540 8032 0.8142430278884463 389.18000173568726
7742 9632 0.8037790697674418 387.20598816871643
9002 11232 0.801460113960114 410.10122990608215


KeyboardInterrupt: 

In [20]:

test_res = test_model(resnet_model, test_loader, end_id=-1)
print(test_res)


29 32 0.90625 9.05814003944397
1402 1632 0.8590686274509803 455.3053822517395
2542 3232 0.786509900990099 453.6626179218292
3870 4832 0.8009105960264901 455.70053911209106
5155 6432 0.8014614427860697 452.3973832130432
6541 8032 0.8143675298804781 447.40111804008484
7742 9632 0.8037790697674418 434.06111693382263
9002 11232 0.801460113960114 433.34816884994507
10232 12832 0.7973815461346634 433.73396492004395
11461 14432 0.7941380266075388 434.91944217681885
12750 16032 0.7952844311377245 448.8045310974121
14105 17632 0.7999659709618875 445.8096408843994
15346 19232 0.7979409317803661 444.265291929245
16535 20832 0.7937307987711214 432.8344612121582
17701 22432 0.7890959343794579 430.64603972435
18843 24032 0.7840795605858855 357.1210608482361
19902 25632 0.7764513108614233 350.1976718902588
21000 27232 0.7711515863689776 328.8714301586151
22265 28832 0.7722322419533851 262.5743718147278
23409 30432 0.7692231861198738 253.9126331806183
24483 32032 0.7643294205794205 258.99896812438965
